In [1]:
import pandas as pd
import pyodbc
import re

In [2]:
trans_type_list = { 
    'deposits' : [
        'DIRECT DEPOSIT',
        'DEPOSIT', 
        'M&T ATM CHECK DEPOSIT',
        'M&T ATM CASH DEPOSIT',
        'MOBILE DEPOSIT',
        'VISA PURCHASE CREDIT'
    ],

    'purchases_payments' : [
        'CHECK PAID',
        'VISA PURCHASE',
        'CARD PURCHASE',
        'VISA PURCHASE (RECURRING)',
        'VISA PIN PURCHASE',
        'PURCHASE WITH CASH BACK',
        'VISA PIN PURCHASE WITH CASH BACK',
        'WEB BILL PAYMENT FROM CHECKING',
        'WEB PAYMENT TO CREDIT CARD',
        'WEB PAYMENT',
        'ACH MISCELLANEOUS DEBIT',
        'AUTO PAYMENT TO M&T BANK'    
    ],

    'withdrawals' : [
        'PREAUTHORIZED WITHDRAWAL',
        'QUICK BANK WITHDRAWAL',
        'ATM WITHDRAWAL',
        'COUNTER WITHDRAWAL'  
    ],

    'transfers' : [
        'WEB TRANSFER TO CHECKING',
        'WEB TRANSFER FROM CHECKING',
        'VISA MONEY TRANSFER',
        'TELLER TRANSFER CREDIT',
        'INCOMING FEDWIRE FUNDS TRANSFER        &',
        'TELLER TRANSFER DEBIT',
        'QUICKBANK TXF FROM SAV TO DDA',
        r'ZELLE P2P PAYMENT SENT                 \\',
        r'ZELLE P2P PAYMENT RECEIVED             \\',
        'CREDIT LINE TRANSFER-M&T BANK'    
    ],

    'fees' : [
        'NON-M&T ATM FEE',
        'INTERNATIONAL TRANSACTION FEE',
        'WAIVE ADD-ON FEE: NON M&T ATM FEE WAIVER',
        'INSUFFICIENT FUNDS FEE',
        'M&T ADD-ON FEE: NON M&T ATM FEE REFUND',
        'WAIVE ADD-ON FEE: NON M&T ATM FEE REFUND',
        'M&T ADD-ON FEE: CHKS/SAFE DEPOSIT BOX',
        'WAIVE ADD-ON FEE: CHKS/SAFE DEPOSIT BOX',
        'M&T ADD-ON FEE: NON M&T ATM FEE WAIVER',
        'EFT SERVICE CHARGE',
        'NON-M&T ATM FEE REV',
        'M&T ADD-ON FEE: OVERDRAFT TRANSFERS',
        'WAIVE ADD-ON FEE: OVERDRAFT TRANSFERS',
        'BUNDLE FEE WAIVER',
        'OVERDRAFT BALANCE EXCESS FEE',
        'SERVICE CHARGE',
        'ATM SURCHARGE REBATE',
        'CHECK VENDOR CHARGE',
        'REVERSE MONTHLY SERVICE CHARGE',
        'REVERSE INSUFFICIENT FUNDS FEE',
        'MONTHLY SERVICE CHARGE',
        'SERVICE CHG WAIVE- RELATIONSHIP PRICING'    
    ],

    'misc' : [
        'ACH CONVERTED CHECK',
        'OVERDRAFT TRANSFER FROM CREDIT CARD',
        'FEDERAL WITHHOLDING ON INTEREST PAYMENT',
        'OD PROTECTION FUND TRANSFER',
        'REVERSE CARD PURCHASE',
        'VISA PIN CREDIT VOUCHER/MERCHANDISE RET',
        'DEBIT CARD DISPUTE PROVISIONAL CREDIT',
        'NSF ITEM PRESENTED NOTATION',
        'CLOSEOUT',
        'INTEREST PAYMENT'  
    ]
}

load_order = [1,2]

metrics = ['transaction_amt', 'trans_post_ledger_bal_amt']

#string and lists to build
loop_string = ''
int_cols = []
float_cols = []

#create transaction counts
for j in load_order:
    loop_string += ",sum(case when cal1.load_order = {0} then 1 else 0 end) as trans_count_total_{0}".format(j)
    int_cols.append("trans_count_total_{0}".format(j))
    while j+1 < 3:
        loop_string += ",sum(case when cal1.load_order = {0} then 1 else 0 end) - sum(case when cal1.load_order = {1} then 1 else 0 end) as trans_count_total_dif".format(j,j+1)
        int_cols.append("trans_count_total_dif".format(j,j+1))
        break
    for i in trans_type_list:
        loop_string += ",sum(case when dt.transaction_desc in ({0}) and cal1.load_order = {1} then 1 else 0 end) as trans_count_{2}_{1}".format(','.join(["'"+l+"'" for l in trans_type_list[i]]) ,j,re.sub(r'\W+', '_', i).strip())
        int_cols.append("trans_count_{1}_{0}".format(j,re.sub(r'\W+', '_', i).strip()))
        while j+1 < 3:
            loop_string += ",sum(case when dt.transaction_desc in ({0}) and cal1.load_order = {1} then 1 else 0 end) - sum(case when dt.transaction_desc in ({0}) and cal1.load_order = {3} then 1 else 0 end) as trans_count_{2}_dif".format(','.join(["'"+l+"'" for l in trans_type_list[i]])  ,j,re.sub(r'\W+', '_', i).strip(),j+1)
            int_cols.append("trans_count_{0}_dif".format(re.sub(r'\W+', '_', i).strip()))
            break

#create averaging metrics
for j in load_order:
    for k in metrics:        
        loop_string += ",avg(case when cal1.load_order = {0} then dt.{1} else 0 end) as {1}_{0}".format(j,k)
        float_cols.append("{1}_{0}".format(j,k))
        while j+1 < 3:
            loop_string += ",avg(case when cal1.load_order = {0} then dt.{2} else 0 end) - sum(case when cal1.load_order = {1} then dt.{2} else 0 end) as {2}_dif".format(j,j+1,k)
            float_cols.append("{0}_dif".format(k))
            break
        for i in trans_type_list:
            loop_string += ",avg(case when dt.transaction_desc in ({1}) and cal1.load_order = {2} then dt.{0}  else 0 end) as {0}_{3}_{2}".format(k,','.join(["'"+l+"'" for l in trans_type_list[i]]) ,j,re.sub(r'\W+', '_', i).strip())
            float_cols.append("{0}_{2}_{1}".format(k,j,re.sub(r'\W+', '_', i).strip()))
            while j+1 < 3:
                loop_string += ",avg(case when dt.transaction_desc in ({1}) and cal1.load_order = {2} then dt.{0}  else 0 end) - avg(case when dt.transaction_desc in ({1}) and cal1.load_order = {4} then dt.{0}  else 0 end) as {0}_{3}_dif".format(k,','.join(["'"+l+"'" for l in trans_type_list[i]]) ,j,re.sub(r'\W+', '_', i).strip(),j+1)
                float_cols.append("{0}_{1}_dif".format(k,re.sub(r'\W+', '_', i).strip()))
                break

In [ ]:
print(loop_string)

### Define Query

In [7]:
queryfromProd = r"""

with cte1 as (
select
        businessmonthbegin
        ,businessmonthend
        ,bcal.as_of_date
        ,year_val
        ,month_val
        , rw-1 as load_order
    from 
        (select 
        as_of_date
        ,row_number() over(order by as_of_date desc) as rw
        from
            (select distinct trunc(business_date, 'month')  as as_of_date
            from p_cdp_v4.deposit_transaction_monthly) a
        ) b
    inner join
        (select distinct
        to_date(calendar_date) as as_of_date
        ,businessmonthbegin
        ,businessmonthend
        ,year(businessmonthbegin) as year_val
        ,month(businessmonthbegin ) as month_val
        from p_cdp_v4.businesscalendar bci) bcal
        on bcal.as_of_date = b.as_of_date),

cal1 as (
    select 
        to_date(businessmonthbegin) as businessmonthbegin
        ,to_date(businessmonthend) as businessmonthend
        ,cast(regexp_replace(cast(as_of_date as char(10)),'-','') as int) as as_of_date
        ,year_val
        ,month_val
        ,load_order
    from cte1 
    order by load_order asc, businessmonthend asc limit 3)
 
SELECT
     p3.party_key
    ,datediff(p3.as_of_date , p3.birth_dt) /365 as person_age
    ,case when p3.gender_cd in ('5', 'Male') then 1 else 0 end as male
    ,case when p3.gender_cd in ('6', 'Female') then 1 else 0 end as female
    ,datediff(p3.as_of_date, p3.account_open_dt) as account_age_days
    ,coalesce(case when p3.account_lob_desc like 'Closed%' then 1 else 0 end, 0) as Closed_within_next_month
    --aggregations
    ,dtm.*

FROM 
    (select 
    dt.account_key
    {0}
    from
    p_cdp_v4.deposit_transaction_monthly dt
    inner join (
        select 
        cal1.businessmonthend
        , cal1.businessmonthbegin
        , load_order 
        from 
        cal1 
        where cal1.load_order in (1,2))cal1
        on ((to_date(dt.transaction_post_dt) >= cal1.businessmonthbegin and to_date(dt.transaction_post_dt) <= cal1.businessmonthend
        and cal1.load_order = 1)
        or
        (to_date(dt.transaction_post_dt) >= cal1.businessmonthbegin and to_date(dt.transaction_post_dt) <= cal1.businessmonthend
        and cal1.load_order = 2))
    group by dt.account_key
    )dtm
    
inner join 
    (select 
    p2.party_key
    ,p2.gender_cd
    ,p2.birth_dt
    ,p2.as_of_date
    ,p2.account_key
    ,ma2.account_lob_desc --this is the 1 month later status
    ,ma.account_open_dt
    
    from
        (select
        p.party_key
        ,p.gender_cd
        ,p.birth_dt
        ,p.business_date as as_of_date
        ,dense_rank() over(partition by p.party_key order by p.as_of_date desc) as rw
        ,pa.account_key
        from
        p_cdp_v4.party p
        inner join 
            (select
            pa.party_key
            ,pa.account_key
            ,as_of_date
            from
            p_cdp_v4.party_account pa
            where pa.customer_ind = 'Y'
            ) pa
        on pa.party_key = p.party_key
        and p.as_of_date = pa.as_of_date
        where p.party_type_desc = 'Individual' 
        and--exclude deceased
        trim(p.deceased_ind) = '' 
        and --exclude deceased
        p.deceased_dt is null
        and business_date 
            <=(select 
                cal1.businessmonthend
                from 
                cal1 
                where cal1.load_order in (1) limit 1)
        ) p2
        inner join 
            (select 
            ma.account_key
            ,ma.source_product_cd
            ,ma.account_lob_desc 
            ,ma.account_open_dt
            ,business_date as as_of_date
            -- ,dense_rank() over(partition by ma.account_key order by ma.as_of_date desc) as rw
            from
            p_cdp_v4.master_account ma
            where
            ma.source_product_cd = 'DDA'
            and ma.account_lob_desc not like 'Closed%'
            and ma.source_subproduct_desc in
                (
                'CORP CHEKING WITH INTEREST',
                'M&T SELECT',
                'BROKERAGE CHECKING ACCOUNT',
                'BUSINESS CHECKING I',
                'STUDENT CHECKING',
                'MMDA INV-OWNED ESCROW DEPOSITS',
                'FREE BUSINESS CHECKING',
                'CORPORATE NOT - FOR - PROFIT N.O.W.',
                'BUSINESS CHECKING II w/interest',
                'M&T SELECT WITH INTEREST',
                'MMDA INVESTOR P&I DEPOSITS',
                'PMA ACCOUNT',
                'M&T DIRECT CHECKING',
                'CORPORATE CHECKING',
                'WORRY FREE CHECKING',
                'MYCHOICE PREMIUM CHECKING',
                'M&T FIRST',
                'NON-PROFIT CHECKING',
                'MYCHOICE CHECKING',
                'MUNICIPAL CHECKING WITH INTEREST',
                'COMMERCIAL CHECKING',
                'POWER CHECKING',
                'M&T CLASSIC CHECKING WITH INTEREST',
                'EZCHOICE CHECKING',
                'M&T BANK SWEEP DEPOSIT ACCOUNT',
                'FREE CHECKING',
                'MYCHOICE PLUS CHECKING',
                'BUSINESS CHECKING II',
                'CLASSIC CHECKING',
                'BUSINESS CHECKING I w/interest',
                'BUSINESS PROCHECKING',
                'COLLEGE CHECKING CHECKING',
                'MYCHOICE PLUS CHECKING WITH INTEREST',
                'NON-PROFIT & SOLE PROPRIETOR N.O.W.',
                'M&T FIRST WITH INTEREST',
                'BizFlex Checking',
                'ADVANCED BUSINESS CHECKING',
                'INTERNAL OPERATING',
                'M&T @ WORK CHECKING',
                'BASIC BANKING',
                'BUSINESS FLEXCHECKING'
                )
            ) ma
        on ma.account_key = p2.account_key
        and p2.as_of_date = ma.as_of_date
        and p2.rw = 1
        
        --get account statuses 1 month later 
        left join
            (select 
            * 
            from 
                (select 
                ma.account_key
                ,ma.account_lob_desc 
                -- ,as_of_date
                ,dense_rank() over(partition by ma.account_key order by as_of_date desc) as rwma2
                from
                p_cdp_v4.master_account ma
                where
                ma.source_subproduct_desc in
                    (
                    'CORP CHEKING WITH INTEREST',
                    'M&T SELECT',
                    'BROKERAGE CHECKING ACCOUNT',
                    'BUSINESS CHECKING I',
                    'STUDENT CHECKING',
                    'MMDA INV-OWNED ESCROW DEPOSITS',
                    'FREE BUSINESS CHECKING',
                    'CORPORATE NOT - FOR - PROFIT N.O.W.',
                    'BUSINESS CHECKING II w/interest',
                    'M&T SELECT WITH INTEREST',
                    'MMDA INVESTOR P&I DEPOSITS',
                    'PMA ACCOUNT',
                    'M&T DIRECT CHECKING',
                    'CORPORATE CHECKING',
                    'WORRY FREE CHECKING',
                    'MYCHOICE PREMIUM CHECKING',
                    'M&T FIRST',
                    'NON-PROFIT CHECKING',
                    'MYCHOICE CHECKING',
                    'MUNICIPAL CHECKING WITH INTEREST',
                    'COMMERCIAL CHECKING',
                    'POWER CHECKING',
                    'M&T CLASSIC CHECKING WITH INTEREST',
                    'EZCHOICE CHECKING',
                    'M&T BANK SWEEP DEPOSIT ACCOUNT',
                    'FREE CHECKING',
                    'MYCHOICE PLUS CHECKING',
                    'BUSINESS CHECKING II',
                    'CLASSIC CHECKING',
                    'BUSINESS CHECKING I w/interest',
                    'BUSINESS PROCHECKING',
                    'COLLEGE CHECKING CHECKING',
                    'MYCHOICE PLUS CHECKING WITH INTEREST',
                    'NON-PROFIT & SOLE PROPRIETOR N.O.W.',
                    'M&T FIRST WITH INTEREST',
                    'BizFlex Checking',
                    'ADVANCED BUSINESS CHECKING',
                    'INTERNAL OPERATING',
                    'M&T @ WORK CHECKING',
                    'BASIC BANKING',
                    'BUSINESS FLEXCHECKING'
                    )
                and
                ma.account_lob_desc like'Closed%'
                and
                ma.source_product_cd = 'DDA'
                and
                business_date
                    <=(select 
                        cal1.businessmonthend
                        from 
                        cal1 
                        where cal1.load_order in (0) limit 1)
                ) ma2
                where ma2.rwma2 = 1
            )ma2
        on ma2.account_key = ma.account_key
        and ma2.rwma2 = 1
    ) p3
on p3.account_key = dtm.account_key

order by 
    coalesce(case when p3.account_lob_desc like 'Closed%' then 1 else 0 end, 0) desc
    

""".format(loop_string)

In [8]:
#don't chnage the connection string
prod_connect_string =  'redacted for security'                        
                        
connProd = pyodbc.connect(prod_connect_string,autocommit=True)

In [9]:
%timeit
write_header = True
directory = r'data'
file = r'\deposit_data_full_condensed.csv'
df = pd.read_sql(queryfromProd, connProd, chunksize= 10000)
for chunk in df:
    chunk.to_csv(directory + file, 
                              sep=',', 
                              index = False,
                              header= write_header,
                              encoding = 'utf-8',
                              mode='a')
    write_header = False

In [26]:
df_partial_import = pd.read_csv(directory+file, dtype={**{i: 'int32' for i in int_cols} , **{i: 'float32' for i in float_cols}})

In [33]:
df_partial_import

,party_key,person_age,male,female,account_age_days,closed_within_next_month,account_key,trans_count_total_1,trans_count_total_dif,trans_count_deposits_1,...,transaction_amt_transfers_2,transaction_amt_fees_2,transaction_amt_misc_2,trans_post_ledger_bal_amt_2,trans_post_ledger_bal_amt_deposits_2,trans_post_ledger_bal_amt_purchases_payments_2,trans_post_ledger_bal_amt_withdrawals_2,trans_post_ledger_bal_amt_transfers_2,trans_post_ledger_bal_amt_fees_2,trans_post_ledger_bal_amt_misc_2
0,46531765,49.106849,1,0,90,1,45269591,128,-8,5,...,0.000000,0.056818,0.830720,198.093521,14.256288,157.188217,14.905378,0.000000,5.370076,3.113864
1,47165428,60.095890,1,0,120,1,45047095,1,-4,0,...,0.000000,0.000000,0.000000,1480.333374,0.000000,0.000000,1174.333374,0.000000,0.000000,0.000000
2,37451305,32.739726,0,0,1084,1,37156147,5,0,3,...,0.000000,0.000000,0.000000,52.500000,31.000000,0.000000,21.500000,0.000000,0.000000,0.000000
3,46542092,69.550685,1,0,195,1,44386125,1,-1,0,...,0.000000,0.000000,0.043333,5201.096680,0.000000,2600.526611,0.000000,0.000000,0.000000,2600.570068
4,39019005,89.726027,1,0,4221,1,35940050,11,-2,1,...,0.000000,0.412500,0.014167,24166.349609,1879.345459,9264.235352,7461.556152,0.000000,3707.396240,1853.815430
5,39450691,49.917808,1,0,7104,1,34885009,1,0,0,...,0.000000,2.475000,0.000000,-2.475000,0.000000,0.000000,0.000000,0.000000,-2.475000,0.000000
6,37269847,30.542466,0,1,153,1,44780995,1,0,0,...,0.000000,0.000000,0.040000,5150.165039,0.000000,0.000000,0.000000,0.000000,0.000000,5150.165039
7,45536817,43.720548,0,1,197,1,44342546,0,-2,0,...,1.150000,19.250000,0.000000,-684.260010,0.000000,0.000000,0.000000,-341.554993,-342.704987,0.000000
8,41345999,33.789041,1,0,155,1,44800058,1,-1,0,...,0.000000,4.983333,0.000000,-4.983333,0.000000,0.000000,0.000000,0.000000,-4.983333,0.000000
9,33025145,63.619178,1,0,18381,1,35189583,4,-2,0,...,0.000000,2.990000,0.002000,1312.350952,584.039978,0.000000,0.000000,0.000000,363.408997,182.451996


In [32]:
df_partial_import.closed_within_next_month.mean()

0.005356659738298862

### Look at data to make sure it's what we expect

In [29]:
def df_stats(df):
    print(df.dtypes)
    print('_'*100)
    # dataframe.size 
    size = df.size 

    # dataframe.shape 
    shape = df.shape 

    # dataframe.ndim 
    df_ndim = df.ndim 

    #df mem usage
    df_memu = df.memory_usage() 


    # printing size and shape 
    print("Size = {}\n\
    Shape = {}\n\
    Shape[0] x Shape[1] = {}\n\
    Memory Usage = \n{}". 
    format(size, shape, shape[0]*shape[1], df_memu)) 

In [30]:
df_stats(df_partial_import)

party_key                                             int64
person_age                                          float64
male                                                  int64
female                                                int64
account_age_days                                      int64
closed_within_next_month                              int64
account_key                                           int64
trans_count_total_1                                   int32
trans_count_total_dif                                 int32
trans_count_deposits_1                                int32
trans_count_deposits_dif                              int32
trans_count_purchases_payments_1                      int32
trans_count_purchases_payments_dif                    int32
trans_count_withdrawals_1                             int32
trans_count_withdrawals_dif                           int32
trans_count_transfers_1                               int32
trans_count_transfers_dif               

In [35]:
df_partial_import.to_pickle(r'data/deposit_data_consensed.pkl')